# Imports

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
tf.__version__

'1.15.3'

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [4]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

# Download the Flowers Dataset using TF Datasets

In [ ]:
(training_set, validation_set), dataset_info = tfds.load('tf_flowers', split=['train[:70%]', 'train[70%:]'], with_info=True, as_supervised=True)

# Print Information about Dataset 

In [ ]:
num_classes = dataset_info.features['label'].num_classes 

num_training_examples = 0
num_validation_examples = 0

for example in training_set:
  num_training_examples += 1
for example in validation_set:
  num_validation_examples += 1

print('Total Number of Classes: {}'.format(num_classes))
print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {} \n'.format(num_validation_examples))

In [ ]:
for i, example in enumerate(training_set.take(5)):
  print('Image {} shape: {} label: {}'.format(i+1, example[0].shape, example[1]))

# Reformat Images and Create Batches

In [ ]:
IMAGE_RES = 224

def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32

train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1) 

validation_batches = validation_set.map(format_image).batch(BATCH_SIZE).prefetch(1)

# Create a Feature Extractor

In [ ]:
URL = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'
feature_extractor = hub.KerasLayer(URL, input_shape=(IMAGE_RES, IMAGE_RES, 3))

# Freeze the Pre-Trained Model

In [ ]:
feature_extractor.trainable = False

# Attach a classification head

In [ ]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(num_classes)
])
model.summary()


# Train the model

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

EPOCHS = 6

history = model.fit(train_batches, epochs=EPOCHS, validation_data=validation_batches)

#  Plot Training and Validation Graphs

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss)
plt.plot(epochs_range, val_loss)
plt.title('Training and Validation Loss')
plt.show()

# Check Predictions

In [ ]:
class_names = np.array(dataset_info.features['label'].names)
print(class_names)


# Create an Image Batch and Make Predictions

In [ ]:
image_batch, label_batch = next(iter(train_batches))

image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()

predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]


# Print True Labels and Predicted Indices

In [ ]:
print(predicted_class_names)

# Plot Model Predictions

In [ ]:
plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")